In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import re
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
house_address_all = []
house_details_all = []
house_address = []
house_details = []
postal_code_array = []
latitude_array = []
longitude_array = []
year_built_array = []
payment_monthly = []
lot_size = []
basement = []
garage = []
mlsa = []
mls_array = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #time.sleep(5)
    
    for each in soup.find_all(class_= "card-text ng-star-inserted" ):
        try: 
            mls = each.find(class_='mlsNumber').text.replace('MLS® #: ', '')
            mls_array.append(mls)
        except:
            mls_array.append(np.nan)

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address_all.append(address.text)
    except:
        house_address_all.append('None')
        
    #time.sleep(5)
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details_all.append(detail.text)
    except:
        house_details_all.append('None')
        
    #time.sleep(5)
        
    house_details_more=[]
    url_house = []
        
    try:
        for detail in soup.find_all('a'):
            house_details_more.append(detail.get('href'))
    except:
        house_details_more.append('None')
    
    #time.sleep(5)

    for i in house_details_more:
        if '/ab/calgary-real-estate/' in str(i):
            link = 'https://www.remax.ca' + str(i)
            url_house.append(link)
            
    #time.sleep(5)
    
    for i in url_house:
    
        time.sleep(5)

        page = requests.get(i)
        soup = BeautifulSoup(page.text, "html.parser")

        lat_all = soup.find_all('script')[3]
        
        string = ', '.join(lat_all)
        pc = "postalCode"
        position_pc = string.find(pc)
        pcd = string[(position_pc+14):(position_pc+20)]
        postal_code_array.append(pcd)

        lat = "latitude"
        position_lat = string.find(lat)
        latd = string[(position_lat+11):(position_lat+20)]
        latitude_array.append(latd)

        long = "longitude"
        position_long = string.find(long)
        longd = string[(position_long+12):(position_long+23)]
        longitude_array.append(longd)

        house_address = []

        #time.sleep(5)

        try:
            addresses = soup.find_all('section', class_='property-details')
            for address in addresses:
                house_address.append(address.text)
        except:
            house_address.append('None')

        #time.sleep(5)

        string = ', '.join(house_address)
        s_year = "Year Built: "
        if s_year in string:            
            position_year = string.find(s_year)
            year_built = string[(position_year+12):(position_year+16)]
            year_built_array.append(year_built)
        else:
            year_built_array.append(np.nan)
        
        s_lot = "Lot Size: "
        if s_lot in string:
            try:
                start = string.rindex(s_lot) + len(s_lot)
                end = string.rindex("Occupancy", start )
                lot_size.append(string[start:end])
            except:
                lot_size.append(np.nan)
        else:
            lot_size.append(np.nan)
        
        basement_yn = "Basement"
        if basement_yn in string:
            basement.append(1)
        else:
            basement.append(0)
            
        garage_yn = "Garage"
        if garage_yn in string:
            garage.append(1)
        else:
            garage.append(0)
        
        try:
            payment = soup.find(class_="payment-amount").text
            payment_monthly.append(payment)
        except:
            payment_monthly.append(np.nan)
        
        #result = re.search('MLSÂ® #: (.*)Ownership', string)
        #result_app = result.group(1)
        #mlsa.append(result.group(1))
        
        mlsa_number = "MLSÂ® #: "
        try:
            start = string.rindex(mlsa_number) + len(mlsa_number)
            end = string.rindex("Ownership", start )
            mlsa.append(string[start:end])
        except:
            #mlsa_pos = string.find(mlsa_number)
            #mlsa_temp = string[(mlsa_pos+8):(mlsa_pos+16)]
            mlsa.append(np.nan)

        #time.sleep(5)

In [3]:
latitude_array[:5]

['51.091129', '50.975174', '51.042621', '51.021818', '51.142426']

In [4]:
longitude_array[:5]

['-113.956141', '-114.073925', '-114.13671\n', '-114.095907', '-114.180925']

In [5]:
postal_code_array[:5]

['T1Y4E8', 'T2V2W3', 'T3C3X6', 'T2T3X9', 'T3G 4J']

In [6]:
mls_array_df_temp = pd.DataFrame(mls_array)
mls_array_df_temp.head()

,0
0,MLSÂ® #: A1046765
1,MLSÂ® #: A1045446
2,MLSÂ® #: A1046854
3,MLSÂ® #: A1046690
4,MLSÂ® #: A1044809


In [7]:
mls_array_df = mls_array_df_temp[0].str.split("MLSÂ® #: ", 1, expand=True)
mls_array_df[1].head()

0    A1046765
1    A1045446
2    A1046854
3    A1046690
4    A1044809
Name: 1, dtype: object

In [8]:
address_df = pd.DataFrame(house_address_all)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")
new_df["price"] = pd.to_numeric(new_df["price"])

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"7172 TEMPLE DR NE, Calgary, AB, T1Y 4E8",339900
1,"611 - 8880 HORTON RD SW, Calgary, AB, T2V 2W3",235000
2,"402 - 77 SPRUCE PL SW, Calgary, AB, T3C 3X6",299888
3,"3710 14A ST SW, Calgary, AB, T2T 3X9",949900
4,"155 Citadel Drive NW, Calgary, AB, T3G 4J5",334900


In [9]:
final_df_pc = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df_pc.head()

,0,1
0,7172 TEMPLE DR NE,T1Y 4E8
1,611 - 8880 HORTON RD SW,T2V 2W3
2,402 - 77 SPRUCE PL SW,T3C 3X6
3,3710 14A ST SW,T2T 3X9
4,155 Citadel Drive NW,T3G 4J5


In [10]:
final_df_pc_clean = final_df_pc[1].str.replace(" ", "")
#final_df_pc_clean = final_df_pc[1]
final_df_pc_clean.head()

0    T1Y4E8
1    T2V2W3
2    T3C3X6
3    T2T3X9
4    T3G4J5
Name: 1, dtype: object

In [11]:
df_add = pd.concat([new_df, final_df_pc_clean, final_df_pc[0], mls_array_df[1]], axis=1)
del df_add[2]
df_add.columns = ["price", "postal_code", "address", "mls"]
df_add.head()

,price,postal_code,address,mls
0,339900,T1Y4E8,7172 TEMPLE DR NE,A1046765
1,235000,T2V2W3,611 - 8880 HORTON RD SW,A1045446
2,299888,T3C3X6,402 - 77 SPRUCE PL SW,A1046854
3,949900,T2T3X9,3710 14A ST SW,A1046690
4,334900,T3G4J5,155 Citadel Drive NW,A1044809


In [12]:
details = pd.DataFrame(house_details_all)
details_df_temp = details[0].str.split('|', expand=True)
details_df_temp.head()

,0,1,2,3
0,5 bed,2 + 1 bath,1066 sqft,house
1,1 bed,1 bath,723 sqft,condo
2,2 bed,2 bath,889 sqft,condo
3,4 bed,3 + 1 bath,1845 sqft,house
4,3 bed,2 + 1 bath,1389 sqft,townhouse


In [13]:
details_df_bed = details_df_temp[0].str.replace(' bed', '')
details_df_bath = details_df_temp[1].str.replace(' bath', '')
details_df_area = details_df_temp[2].str.replace(' sqft', '')

In [14]:
details_df_bath_all = details_df_bath.str.split('+', expand=True)
details_df_bath_full = details_df_bath_all[0]
details_df_bath_half = details_df_bath_all[1]

In [15]:
details_df_bed = details_df_bed.replace('N/A', np.nan)
details_df_bed = pd.to_numeric(details_df_bed)
details_df_area = details_df_area.replace('N/A', np.nan)
details_df_area = pd.to_numeric(details_df_area)
details_df_bath_full = details_df_bath_full.replace('N/A', np.nan)
details_df_bath_full = pd.to_numeric(details_df_bath_full)
details_df_bath_half = details_df_bath_half.replace('N/A', np.nan)
details_df_bath_half = pd.to_numeric(details_df_bath_half)

In [16]:
data = {'bed':details_df_bed, 'full_bath':details_df_bath_full, 'half_bath':details_df_bath_half,
       'property_area':details_df_area, 'property_type':details_df_temp[3]}

In [17]:
details_df = pd.DataFrame(data)
details_df.head()

,bed,full_bath,half_bath,property_area,property_type
0,5,2,1.0,1066,house
1,1,1,NaN,723,condo
2,2,2,NaN,889,condo
3,4,3,1.0,1845,house
4,3,2,1.0,1389,townhouse


In [18]:
calgary_df_dup = pd.concat([df_add, details_df], axis=1)
calgary_df = calgary_df_dup.drop_duplicates()
calgary_df.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type
0,339900,T1Y4E8,7172 TEMPLE DR NE,A1046765,5,2,1.0,1066,house
1,235000,T2V2W3,611 - 8880 HORTON RD SW,A1045446,1,1,NaN,723,condo
2,299888,T3C3X6,402 - 77 SPRUCE PL SW,A1046854,2,2,NaN,889,condo
3,949900,T2T3X9,3710 14A ST SW,A1046690,4,3,1.0,1845,house
4,334900,T3G4J5,155 Citadel Drive NW,A1044809,3,2,1.0,1389,townhouse


In [19]:
len(calgary_df)

20

In [20]:
calgary_df.to_csv('data/calgary_listings-a.csv', index=False)

## Cleanup Postal Codes, Latitude, Longitude and Year Built

postal_codes_messy = pd.DataFrame(postal_code_array, columns=["postal_code"])

postal_codes_messy.head()

postal_code_array_clean = postal_codes_messy["postal_code"].str.replace('"', '')
postal_code_array_clean = postal_code_array_clean.str.replace(" ", "")

In [21]:
latitude_array_messy = pd.DataFrame(latitude_array, columns=["latitude"])

In [23]:
#latitude_array_messy

In [24]:
latitude_array_messy = latitude_array_messy["latitude"].str.replace(',', '')
latitude_array_messy.head()

0    51.091129
1    50.975174
2    51.042621
3    51.021818
4    51.142426
Name: latitude, dtype: object

In [25]:
longitude_array_messy = pd.DataFrame(longitude_array, columns=["longitude"])

In [26]:
longitude_array_messy = longitude_array_messy["longitude"].str.replace(',', '')
longitude_array_messy.head()

0     -113.956141
1     -114.073925
2    -114.13671\n
3     -114.095907
4     -114.180925
Name: longitude, dtype: object

<div class="alert alert-danger">
    <b>Unused</b>
    
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])<br>
year_built_clean = year_built_messy["year_built"].str.replace('', np.nan)<br>
year_built_num = year_built_messy.astype(int)

year_built_num.head()
</div>

In [27]:
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])
#year_built_messy = pd.to_numeric(year_built_messy)
year_built_messy.head()

,year_built
0,1979
1,2010
2,2008
3,2013
4,1995


In [28]:
year_built_clean = year_built_messy["year_built"].replace('', np.nan)

In [29]:
year_built_clean.head(11)

0     1979
1     2010
2     2008
3     2013
4     1995
5     1977
6     1997
7     2004
8     2017
9     1968
10    1979
Name: year_built, dtype: object

In [30]:
year_built_digits = []

for i in year_built_clean:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        #year_built_clean.replace(i, np.nan)
        year_built_digits.append(np.nan)
    if ((str(i) == ' 9 2') or (str(i) == ' 8 2') or (str(i) == ' 2 2') or (str(i) == 'nasq') or (str(i) == ' 6 R') or (str(i) == ' 6 S')):
        year_built_digits.append(np.nan)
    else:
        year_built_digits.append(i)

In [31]:
year_built_digits[:11]

['1979',
 '2010',
 '2008',
 '2013',
 '1995',
 '1977',
 '1997',
 '2004',
 '2017',
 '1968',
 '1979']

In [32]:
year_built_num = pd.to_numeric(year_built_digits)
year_built_num[:10]

array([1979, 2010, 2008, 2013, 1995, 1977, 1997, 2004, 2017, 1968],
      dtype=int64)

In [33]:
year_built_num_pd = pd.DataFrame(year_built_num)

In [34]:
years_old = 2020 - year_built_num
years_old[:10]

array([41, 10, 12,  7, 25, 43, 23, 16,  3, 52], dtype=int64)

In [35]:
years_old_df = pd.DataFrame(years_old)

In [36]:
latitude_digits = []

for i in latitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        latitude_digits.append(np.nan)
    else:
        latitude_digits.append(i)

In [37]:
longitude_digits = []

for i in longitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        longitude_digits.append(np.nan)
    else:
        longitude_digits.append(i)

In [38]:
latitude_digits = pd.to_numeric(latitude_digits)
longitude_digits = pd.to_numeric(longitude_digits)

In [39]:
payment_monthly_messy = pd.DataFrame(payment_monthly, columns=["monthly_payments"])
payment_monthly_messy.head()

,monthly_payments
0,$1469.77
1,$1016.17
2,$1296.75
3,$4107.48
4,$1448.15


In [40]:
payment_monthly_messy = payment_monthly_messy["monthly_payments"].str.replace('$', '')
payment_monthly_messy.head(10)

0    1469.77
1    1016.17
2    1296.75
3    4107.48
4    1448.15
5     626.57
6    1924.23
7    1426.53
8     972.50
9    1729.22
Name: monthly_payments, dtype: object

In [41]:
payment_monthly_messy = payment_monthly_messy.replace('', np.nan)

In [42]:
payment_monthly_clean = payment_monthly_messy.replace('None', np.nan)
payment_monthly_clean = pd.to_numeric(payment_monthly_clean)

<div class="alert alert-success">
    <b>Guide</b>
    
https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

#### approximate radius of earth in km
R = 6373.0
    
lat1 = radians(52.2296756)<br>
lon1 = radians(21.0122287)

lat2 = radians(52.406374)<br>
lon2 = radians(16.9251681)

dlon = lon2 - lon1<br>
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2<br>
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
    
#### Calgary Downtown coordinates (51.0449485,-114.0739559)
</div>

In [45]:
distance_downtown = []

for i, j in zip(latitude_digits, longitude_digits):
    lat1 = radians(i)
    lon1 = radians(j)
    lat2 = radians(51.0449485)
    lon2 = radians(-114.0739559)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6373.0
    distance = (R * c)
    
    distance_downtown.append(round(distance, 2))

In [46]:
distance_downtown[:5]

[9.71, 7.76, 4.4, 3.0, 13.17]

In [47]:
distance_downtown_df = pd.DataFrame(distance_downtown)

<div class="alert alert-warning">
    <b>Another example</b>
    
import geopy.distance

coords_1 = (52.2296756, 21.0122287)<br>
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)
</div>

In [48]:
more_details = pd.DataFrame([latitude_digits, longitude_digits, year_built_num]).T
#more_details.columns = ["postal_code", "latitude", "longitude", "year_built"]
more_details.head()

,0,1,2
0,51.091129,-113.956141,1979.0
1,50.975174,-114.073925,2010.0
2,51.042621,-114.136710,2008.0
3,51.021818,-114.095907,2013.0
4,51.142426,-114.180925,1995.0


In [49]:
lot_size_df = pd.DataFrame(lot_size)

In [50]:
basement_df = pd.DataFrame(basement)

In [51]:
garage_df = pd.DataFrame(garage)

In [52]:
mls_df = pd.DataFrame(mlsa)

In [53]:
more_df = pd.concat([more_details, years_old_df, distance_downtown_df, payment_monthly_clean, lot_size_df, basement_df, garage_df, mls_df], axis=1)
more_df.columns = ["latitude", "longitude", "year_built", "years_old", "distance_downtown", "monthly_payments", "lot_size", "basement", "garage", "mls"]
more_df.head()

,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage,mls
0,51.091129,-113.956141,1979.0,41,9.71,1469.77,4790,1,1,A1046765
1,50.975174,-114.073925,2010.0,10,7.76,1016.17,NaN,0,1,A1045446
2,51.042621,-114.136710,2008.0,12,4.40,1296.75,NaN,0,0,A1046854
3,51.021818,-114.095907,2013.0,7,3.00,4107.48,3132,1,1,A1046690
4,51.142426,-114.180925,1995.0,25,13.17,1448.15,2314,1,1,A1044809


In [54]:
len(more_df)

60

In [55]:
more_df.to_csv('data/calgary_listings-b.csv', index=False)

In [61]:
result_dup = pd.merge(calgary_df, more_df, on='mls')
result = result_dup.drop_duplicates()
result = result.reset_index(drop=True)
result.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,339900,T1Y4E8,7172 TEMPLE DR NE,A1046765,5,2,1.0,1066,house,51.091129,-113.956141,1979.0,41,9.71,1469.77,4790,1,1
1,235000,T2V2W3,611 - 8880 HORTON RD SW,A1045446,1,1,NaN,723,condo,50.975174,-114.073925,2010.0,10,7.76,1016.17,NaN,0,1
2,299888,T3C3X6,402 - 77 SPRUCE PL SW,A1046854,2,2,NaN,889,condo,51.042621,-114.136710,2008.0,12,4.40,1296.75,NaN,0,0
3,949900,T2T3X9,3710 14A ST SW,A1046690,4,3,1.0,1845,house,51.021818,-114.095907,2013.0,7,3.00,4107.48,3132,1,1
4,334900,T3G4J5,155 Citadel Drive NW,A1044809,3,2,1.0,1389,townhouse,51.142426,-114.180925,1995.0,25,13.17,1448.15,2314,1,1


In [62]:
len(result)

20

In [63]:
result_df_clean = result.sort_values('address').drop_duplicates(subset=['mls']) 
result_df_clean.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
11,597900,T3P0K3,100 EVANSFIELD PL NW,A1046587,4,3,1.0,2467,house,51.177351,-114.110308,2012.0,8,14.94,2585.39,5468,1,1
14,319900,T2W1G1,10304 ELBOW DR SW,A1046733,3,1,NaN,1040,house,50.961657,-114.084377,1961.0,59,9.29,1383.29,5500,1,1
12,349000,T3M1C3,110 CRAMOND PL SE,A1046724,3,2,1.0,1245,house,50.893400,-113.979031,2001.0,19,18.12,1509.12,2669,1,1
7,329900,T3L3C2,15 Toscana Gardens NW,A1046451,2,2,1.0,1345,townhouse,51.121067,-114.254993,2004.0,16,15.22,1426.53,na,0,1
4,334900,T3G4J5,155 Citadel Drive NW,A1044809,3,2,1.0,1389,townhouse,51.142426,-114.180925,1995.0,25,13.17,1448.15,2314,1,1


In [64]:
len(result_df_clean)

20

In [65]:
result_df_clean.to_csv('data/calgary_listings.csv', index=False)

-----------

<div class="alert alert-info">
    <b>UNSED</b>
</div>

In [ ]:
house_details_more=[]
url_house = []
try:
    for detail in soup.find_all('a'):
        house_details_more.append(detail.get('href'))
except:
    house_details_more.append('None')
    
for i in house_details_more:
    if '/ab/calgary-real-estate/' in str(i):
        link = 'https://www.remax.ca' + str(i)
        url_house.append(link)
url_house

In [ ]:
url_house[:5]

In [ ]:
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

for i in url_house[:5]:
    
    time.sleep(5)
    
    page = requests.get(i)
    soup = BeautifulSoup(page.text, "html.parser")
    
    lat_all = soup.find_all('script')[4]
    
    string = ', '.join(lat_all)
    pc = "postalCode"
    position_pc = string.find(pc)
    pcd = string[(position_pc+14):(position_pc+21)]
    PostalCode.append(pcd)

    lat = "latitude"
    position_lat = string.find(lat)
    latd = string[(position_lat+11):(position_lat+20)]
    Latitude.append(latd)

    long = "longitude"
    position_long = string.find(long)
    longd = string[(position_long+12):(position_long+23)]
    Longitude.append(longd)
    
    house_address = []
    
    time.sleep(5)
    
    try:
        addresses = soup.find_all('section', class_='property-details')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
    
    time.sleep(5)
    
    string = ', '.join(house_address)
    s_year = "Year Built"
    position_year = string.find(s_year)
    year_built = string[(position_year+12):(position_year+16)]
    YearBuilt.append(year_built)